In [52]:
import pandas as pd
import dataFile
import getNameNumbers
import sqlalchemy
import mysql.connector
import sqlite3

#
# Read in the CSV file for processing in pandas
#

path = '/home/ec2-user/'
#path = '/Users/jameshayes/'
file_name = 'tempest_temp.csv'
full_file = f'{path}{file_name}'
df = pd.read_csv(full_file, index_col=False)

pd.set_option('display.max_rows', 1440)
pd.set_option('display.max_columns', 35)
pd.set_option('display.width', 1500)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)

max_temp  = (df.sort_values(by='temperature', ascending=False))
max_T = max_temp.iloc[:1]
maxT = max_T['temperature'].values[0]
maxT = round((maxT*1.8) + 32)

min_temp  = (df.sort_values(by='temperature', ascending=True))
min_T = min_temp.iloc[:1]
minT = min_T['temperature'].values[0]
minT = round((minT*1.8) + 32)

max_pres = (df.sort_values(by='pressure', ascending=False))
max_P = max_pres.iloc[:1]
maxP = max_P['pressure'].values[0]

min_pres = (df.sort_values(by='pressure', ascending=True))
min_P = min_pres.iloc[:1]
minP = min_P['pressure'].values[0]

cor_rain = (df.sort_values(by='local_daily_precip_final', ascending=False))
corR_rain = cor_rain.iloc[:1]
corR = corR_rain['local_daily_precip_final'].values[0]
corR = round((corR*0.03937), 2)

tot_rain = df['precip'].sum()
totR = round((tot_rain*0.03937), 2)

strike_distance = (df.sort_values(by='strike_distance', ascending=False))
lightning1 = (df['strike_distance'].between(1,8))
lightning2 = (df['strike_distance'].between(9,16))

df.insert(10,'lightning1',lightning1)
df.insert(11,'lightning2',lightning2)

#Determine if the strike distance is close enough to count as a thunderstorm
x = len(df)
a = 0

while a < x:
        if (df['lightning1'] == True).any():
            q = "Yes"
        else:
            q = "No"
        if (df['lightning2'] == True).any():   
            r1 = "Yes"
        else:
            r1 = "No"
        a += 1    
  
 
strike_count = df['strike_count'].sum()

gg = getNameNumbers.sqlWrite()
dead1, dead2, dead3, date, yesterdayDay, month, year, dead4 = gg[0], gg[1], gg[2], gg[3], gg[4], gg[5], gg[6], gg[7]
print(f'The value of date is : {date}')
yesterday  = (date - 1)
print(f'The value of yesterday is : {yesterday}')
print(f'This year is : {year}')

df2 = pd.DataFrame(columns = ['Year', 'Month', 'Date', 'High', 'Low', 'totR', 'corR', 'Lightning1_5', 'Lightning6_10'])
newRow = pd.DataFrame({'Year': year, 'Month': month, 'Date': yesterday, 'High': maxT, 'Low': minT, 'totR' : totR, 'corR' : corR, 'Lightning1_5': q, 'Lightning6_10' : r1}, index = [date])
df2 = pd.concat([newRow, df2[:]]).reset_index(drop = True)
print(df2)

database_username = 'chuckwx'
database_password = 'jfr716!!00'
database_ip       = '3.135.162.69'
database_name     = 'tempestf6'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), connect_args={'connect_timeout': 30})
df2.to_sql(con=database_connection, name='tempestF6', if_exists='append', index = False)

Today is:  2023-08-09
The day of the month is:  9 

This month is : August
This year is:  2023
Number of days this month:  31 

Yesterday was:  2023-08-08
9
The day of the month yesterday was:  8 

Last month was:  July
<class 'datetime.date'>
Last year was:  2022 

Number of days last month:  31 

Last day of the previous month:  31
Last day of the previous year:  31 

The value of date is : 9
The value of yesterday is : 7
This year is : 2023


-1